In [35]:
import os
from finrl import config
from finrl import config_tickers

if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

In [36]:
df = YahooDownloader(start_date = '2008-01-01',
                     end_date = '2021-09-02',
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [37]:


fe = FeatureEngineer(
                    use_technical_indicator=True,
                    use_turbulence=False,
                    user_defined_feature = False)

df = fe.preprocess_data(df)



Successfully added technical indicators


In [38]:
# add covariance matrix as states
df=df.sort_values(['date','tic'],ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
return_list = []

# look back is one year
lookback=252
for i in range(lookback,len(df.index.unique())):
  data_lookback = df.loc[i-lookback:i,:]
  price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
  return_lookback = price_lookback.pct_change().dropna()
  return_list.append(return_lookback)

  covs = return_lookback.cov().values 
  cov_list.append(covs)

  
df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list,'return_list':return_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date','tic']).reset_index(drop=True)

In [39]:
train = data_split(df, '2009-01-01','2020-06-30')

In [40]:
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv


class StockPortfolioEnv(gym.Env):
    """A portfolio allocation environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then 
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step
        

    """
    metadata = {'render.modes': ['human']}

    def __init__(self, 
                df,
                stock_dim,
                hmax,
                initial_amount,
                transaction_cost_pct,
                reward_scaling,
                state_space,
                action_space,
                tech_indicator_list,
                turbulence_threshold=None,
                lookback=252,
                day = 0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low = 0, high = 1,shape = (self.action_space,)) 
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list),self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False     
        self.turbulence_threshold = turbulence_threshold        
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]

        
    def step(self, actions):
        self.terminal = self.day >= len(self.df.index.unique())-1

        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
            plt.plot(df.daily_return.cumsum(),'r')
            plt.savefig('results/cumulative_reward.png')
            plt.close()
            
            plt.plot(self.portfolio_return_memory,'r')
            plt.savefig('results/rewards.png')
            plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))           
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() !=0:
              sharpe = (252**0.5)*df_daily_return['daily_return'].mean()/ \
                       df_daily_return['daily_return'].std()
              print("Sharpe: ",sharpe)
            print("=================================")
            
            return self.state, self.reward, self.terminal,{}

        else:
            weights = self.softmax_normalization(actions) 
            self.actions_memory.append(weights)
            last_day_memory = self.data

            #load next state
            self.day += 1
            self.data = self.df.loc[self.day,:]
            self.covs = self.data['cov_list'].values[0]
            self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values)-1)*weights)
            log_portfolio_return = np.log(sum((self.data.close.values / last_day_memory.close.values)*weights))
            # update portfolio value
            new_portfolio_value = self.portfolio_value*(1+portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])            
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value
            

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False 
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]] 
        return self.state
    
    def render(self, mode='human'):
        return self.state
        
    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator/denominator
        return softmax_output

    
    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']
        
        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [41]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")
tech_indicator_list = ['macd', 'rsi_30', 'cci_30', 'dx_30']
feature_dimension = len(tech_indicator_list)
print(f"Feature Dimension: {feature_dimension}")

Stock Dimension: 28, State Space: 28
Feature Dimension: 4


In [42]:


env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "transaction_cost_pct": 0, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": tech_indicator_list, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-1
    
}

e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)



In [43]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [44]:


agent = DRLAgent(env = env_train)

A2C_PARAMS = {"n_steps": 10, "ent_coef": 0.005, "learning_rate": 0.0004}
model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS)



{'n_steps': 10, 'ent_coef': 0.005, 'learning_rate': 0.0004}
Using cuda device


In [45]:
trained_a2c = agent.train_model(model=model_a2c, tb_log_name='a2c',
                                total_timesteps=40000)

-------------------------------------
| time/                 |           |
|    fps                | 145       |
|    iterations         | 100       |
|    time_elapsed       | 6         |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy_loss       | -39.6     |
|    explained_variance | 0         |
|    learning_rate      | 0.0004    |
|    n_updates          | 99        |
|    policy_loss        | 4.78e+08  |
|    reward             | 2080776.5 |
|    std                | 0.995     |
|    value_loss         | 1.51e+14  |
-------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 190       |
|    iterations         | 200       |
|    time_elapsed       | 10        |
|    total_timesteps    | 2000      |
| train/                |           |
|    entropy_loss       | -39.6     |
|    explained_variance | 0         |
|    learning_rate      | 0.0004    |
|    n_updat

In [46]:
trade = data_split(df,'2020-07-01', '2021-09-02')
e_trade_gym = StockPortfolioEnv(df = trade, **env_kwargs)

In [62]:
! /opt/conda/envs/py39/bin/python -m pip install plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 16.0 MB/s eta 0:00:0000:0100:01


In [63]:
import torch
%matplotlib inlinefrom pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
import pandas as pd
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import objective_functions
unique_tic = trade.tic.unique()
unique_trade_date = trade.date.unique()
import plotly.express as px

In [64]:
import pyfolio
%matplotlib inline
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts

baseline_df = get_baseline(
        ticker="^DJI", 
        start = '2020-07-01',
        end =  '2021-09-01')

baseline_df_stats = backtest_stats(baseline_df, value_col_name = 'close')
baseline_returns = get_daily_return(baseline_df, value_col_name="close")

dji_cumpod =(baseline_returns+1).cumprod()-1

[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (295, 8)
Annual return          0.311845
Cumulative returns     0.374034
Annual volatility      0.140762
Sharpe ratio           2.006165
Calmar ratio           3.491806
Stability              0.950106
Max drawdown          -0.089308
Omega ratio            1.397014
Sortino ratio          2.988706
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.094883
Daily value at risk   -0.016614
dtype: float64


In [72]:
! /opt/conda/envs/py39/bin/python -m pip install PyPortfolioOpt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 1.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 12.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.1/220.1 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.4/298.4 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 22.5 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.6 MB/s eta 0:00:0000:01


In [73]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
import pandas as pd
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import objective_functions
unique_tic = trade.tic.unique()
unique_trade_date = trade.date.unique()

In [74]:
from pyfolio import timeseries

df_daily_return_a2c, df_actions_a2c = DRLAgent.DRL_prediction(model=trained_a2c,
                        environment = e_trade_gym)

time_ind = pd.Series(df_daily_return_a2c.date)
a2c_cumpod =(df_daily_return_a2c.daily_return+1).cumprod()-1
DRL_strat_a2c = convert_daily_return_to_pyfolio_ts(df_daily_return_a2c)

perf_func = timeseries.perf_stats 
perf_stats_all_a2c = perf_func( returns=DRL_strat_a2c, 
                              factor_returns=DRL_strat_a2c, 
                                positions=None, transactions=None, turnover_denom="AGB")


begin_total_asset:1000000
end_total_asset:1410375.1813169278
Sharpe:  2.172836582693974
hit end!


In [75]:
def extract_weights(drl_actions_list):
  a2c_weight_df = {'date':[], 'weights':[]}
  for i in range(len(drl_actions_list)):
    date = drl_actions_list.index[i]
    tic_list = list(drl_actions_list.columns)
    weights_list = drl_actions_list.reset_index()[list(drl_actions_list.columns)].iloc[i].values
    weight_dict = {'tic':[], 'weight':[]}
    for j in range(len(tic_list)):
      weight_dict['tic'] += [tic_list[j]]
      weight_dict['weight'] += [weights_list[j]]

    a2c_weight_df['date'] += [date]
    a2c_weight_df['weights'] += [pd.DataFrame(weight_dict)]

  a2c_weights = pd.DataFrame(a2c_weight_df)
  return a2c_weights

a2c_weights = extract_weights(df_actions_a2c)

In [77]:
def calculate_gradient(model, interpolated_input, actions,  feature_idx, stock_idx, h = 1e-1):
  forward_input = interpolated_input
  forward_input[feature_idx + stock_dimension][stock_idx] += h
  forward_Q = model.policy.evaluate_actions(torch.cuda.FloatTensor(forward_input).reshape(-1,stock_dimension*(stock_dimension + feature_dimension)), torch.cuda.FloatTensor(actions).reshape(-1,stock_dimension))
  interpolated_Q = model.policy.evaluate_actions(torch.cuda.FloatTensor(interpolated_input).reshape(-1,stock_dimension*(stock_dimension + feature_dimension)), torch.cuda.FloatTensor(actions).reshape(-1,stock_dimension))
  forward_Q = forward_Q[0].detach().cpu().numpy()[0]
  interpolated_Q = interpolated_Q[0].detach().cpu().numpy()[0]
  return (forward_Q - interpolated_Q) / h

In [80]:
len(unique_trade_date)

296

In [81]:
import copy 
meta_Q = {"date":[], "feature":[], "Saliency Map":[], "algo":[]}

for algo in {"A2C", "PPO"}:
  if algo == "A2C":
    prec_step = 1e-2
  else:
    prec_step = 1e-1

  model = eval("trained_" + algo.lower())
  df_actions = eval("df_actions_" + algo.lower())
  for i in range(len(unique_trade_date)-1):
    print(f"Unique trade date: {i}")
    date = unique_trade_date[i]
    covs = trade[trade['date'] == date].cov_list.iloc[0]
    features = trade[trade['date'] == date][tech_indicator_list].values # N x K
    actions = df_actions.loc[date].values
   
    for feature_idx in range(len(tech_indicator_list)):
      
      int_grad_per_feature = 0
      for stock_idx in range(features.shape[0]):#N
        print(f"stock index : {stock_idx}")
        int_grad_per_stock = 0
        avg_interpolated_grad = 0
        for alpha in range(1, 51):
          scale = 1/50
          baseline_features = copy.deepcopy(features)
          baseline_noise = np.random.normal(0, 1, stock_dimension)
          baseline_features[:,feature_idx] = [0] * stock_dimension
          interpolated_features = baseline_features + scale * alpha * (features - baseline_features) # N x K
          interpolated_input = np.append(covs, interpolated_features.T, axis = 0)
          interpolated_gradient = calculate_gradient(model, interpolated_input, actions, feature_idx, stock_idx, h = prec_step)[0]
          
          avg_interpolated_grad += interpolated_gradient * scale
        int_grad_per_stock = (features[stock_idx][feature_idx] - 0) * avg_interpolated_grad
        int_grad_per_feature += int_grad_per_stock
      
      meta_Q['date'] += [date]
      meta_Q['algo'] += [algo]
      meta_Q['feature'] += [tech_indicator_list[feature_idx]]
      meta_Q['Saliency Map'] += [int_grad_per_feature]

meta_Q = pd.DataFrame(meta_Q)

Unique trade date: 0
stock index : 0
stock index : 1
stock index : 2
stock index : 3
stock index : 4
stock index : 5
stock index : 6
stock index : 7
stock index : 8
stock index : 9
stock index : 10
stock index : 11
stock index : 12
stock index : 13
stock index : 14
stock index : 15
stock index : 16
stock index : 17
stock index : 18
stock index : 19
stock index : 20
stock index : 21
stock index : 22
stock index : 23
stock index : 24
stock index : 25
stock index : 26
stock index : 27
stock index : 0
stock index : 1
stock index : 2
stock index : 3
stock index : 4
stock index : 5
stock index : 6
stock index : 7
stock index : 8
stock index : 9
stock index : 10
stock index : 11
stock index : 12
stock index : 13
stock index : 14
stock index : 15
stock index : 16
stock index : 17
stock index : 18
stock index : 19
stock index : 20
stock index : 21
stock index : 22
stock index : 23
stock index : 24
stock index : 25
stock index : 26
stock index : 27
stock index : 0
stock index : 1
stock index : 2

KeyboardInterrupt: 

In [82]:
df

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list
0,2008-12-31,3.070357,3.133571,3.047857,2.594392,607541200,AAPL,2,-0.082938,3.106201,2.464247,42.254796,-80.471600,16.129793,2.760714,2.873279,"[[0.0013489695567676136, 0.000428412555243939,...",tic AAPL AMGN AXP ...
1,2008-12-31,57.110001,58.220001,57.060001,42.911240,6287200,AMGN,2,0.160772,43.800974,41.899650,51.060607,51.555637,10.432018,42.109739,41.643845,"[[0.0013489695567676136, 0.000428412555243939,...",tic AAPL AMGN AXP ...
2,2008-12-31,17.969999,18.750000,17.910000,14.690861,9625600,AXP,2,-0.942648,18.787690,12.756088,42.554838,-75.418979,25.776759,15.862963,17.749413,"[[0.0013489695567676136, 0.000428412555243939,...",tic AAPL AMGN AXP ...
3,2008-12-31,41.590000,43.049999,41.500000,32.005890,5443100,BA,2,-0.279799,32.174383,28.867836,47.440243,156.994841,5.366299,30.327214,32.389915,"[[0.0013489695567676136, 0.000428412555243939,...",tic AAPL AMGN AXP ...
4,2008-12-31,43.700001,45.099998,43.700001,29.909035,6277400,CAT,2,0.662261,30.655967,25.713898,51.205309,98.386020,26.331746,26.960316,26.691659,"[[0.0013489695567676136, 0.000428412555243939,...",tic AAPL AMGN AXP ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89315,2021-09-01,159.429993,160.229996,158.210007,153.382690,1133800,TRV,2,2.059329,158.307730,145.924054,55.446787,62.494989,22.987009,149.152018,147.576910,"[[0.00038578115328216146, 0.000101384478109233...",tic AAPL AMGN AXP ...
89316,2021-09-01,416.540009,419.869995,411.239990,407.843781,2034400,UNH,2,1.515093,420.218138,395.008183,53.065126,-4.135050,15.845681,407.074055,401.486494,"[[0.00038578115328216146, 0.000101384478109233...",tic AAPL AMGN AXP ...
89317,2021-09-01,55.000000,55.150002,54.799999,49.855484,12647300,VZ,2,-0.226094,50.958199,49.420654,44.243163,-102.805984,25.659221,50.342786,50.559082,"[[0.00038578115328216146, 0.000101384478109233...",tic AAPL AMGN AXP ...
89318,2021-09-01,50.730000,50.810001,49.480000,46.984314,5212400,WBA,2,0.458375,47.166792,43.282968,54.782459,155.844494,20.132924,44.620924,45.517418,"[[0.00038578115328216146, 0.000101384478109233...",tic AAPL AMGN AXP ...


In [ ]:
import statsmodels.api as sm
meta_score_coef = {"date":[], "coef":[], "algo":[]}

for algo in ["A2C"]:
  if algo == "LR":
    weights = lr_weights
  elif algo == "RF":
    weights = rf_weights
  elif algo == "DT":
    weights = dt_weights
  elif algo == "SVM":
    weights = svm_weights
  elif algo == "A2C":
    weights = a2c_weights
  elif algo == "PPO":
    weights = ppo_weights
  else:
    weights = reference_weights

  for i in range(len(unique_trade_date) - 1):
    date = unique_trade_date[i]
    next_date = unique_trade_date[i+1]
    df_temp = df[df.date==date].reset_index(drop=True)
    df_temp_next = df[df.date==next_date].reset_index(drop=True)
    weight_piece = weights[weights.date == date].iloc[0]['weights']
    piece_return = pd.DataFrame(df_temp_next.return_list.iloc[0].loc[next_date]).reset_index()
    piece_return.columns = ['tic', 'return']
    X = df_temp[['macd','rsi_30', 'cci_30', 'dx_30', 'tic']]
    X_next = df_temp_next[['macd','rsi_30', 'cci_30', 'dx_30', 'tic']]
    piece = weight_piece.merge(X, on = 'tic').merge(piece_return, on = 'tic')
    piece['Y'] = piece['return'] * piece['weight']
    X = piece[['macd','rsi_30', 'cci_30', 'dx_30']]
    X = sm.add_constant(X)
    Y = piece[['Y']]
    model = sm.OLS(Y,X)
    results = model.fit()
    meta_score_coef["coef"] += [(X * results.params).sum(axis = 0)]
    meta_score_coef["date"] += [date]
    meta_score_coef["algo"] += [algo]

meta_score_coef = pd.DataFrame(meta_score_coef)

In [ ]:
performance_score = {"date":[], "algo":[], "score":[]}

for i in range(0, len(unique_trade_date)):
  date_ = unique_trade_date[i]
  if len(meta_score_coef[(meta_score_coef['date'] == date_)]) == 0:
    continue 
  lr_coef = meta_score_coef[(meta_score_coef['date'] == date_) & (meta_score_coef['algo'] == 'LR')]['coef'].values[0][['macd','rsi_30','cci_30','dx_30']].values
  rf_coef = meta_score_coef[(meta_score_coef['date'] == date_) & (meta_score_coef['algo'] == 'RF')]['coef'].values[0][['macd','rsi_30','cci_30','dx_30']].values
  reference_coef = meta_score_coef[(meta_score_coef['date'] == date_) & (meta_score_coef['algo'] == 'Reference Model')]['coef'].values[0][['macd','rsi_30','cci_30','dx_30']].values
  dt_coef = meta_score_coef[(meta_score_coef['date'] == date_) & (meta_score_coef['algo'] == 'DT')]['coef'].values[0][['macd','rsi_30','cci_30','dx_30']].values
  svm_coef = meta_score_coef[(meta_score_coef['date'] == date_) & (meta_score_coef['algo'] == 'SVM')]['coef'].values[0][['macd','rsi_30','cci_30','dx_30']].values

  saliency_coef_a2c = meta_Q[(meta_Q['date'] == date_) & (meta_Q['algo'] == "A2C")]['Saliency Map'].values
  saliency_coef_ppo = meta_Q[(meta_Q['date'] == date_) & (meta_Q['algo'] == "PPO")]['Saliency Map'].values

  lr_score = np.corrcoef(lr_coef, reference_coef)[0][1]
  rf_score = np.corrcoef(rf_coef, reference_coef)[0][1]
  dt_score = np.corrcoef(dt_coef, reference_coef)[0][1]
  svm_score = np.corrcoef(svm_coef, reference_coef)[0][1]
  saliency_score_a2c = np.corrcoef(saliency_coef_a2c, reference_coef)[0][1]
  saliency_score_ppo = np.corrcoef(saliency_coef_ppo, reference_coef)[0][1]

  for algo in ["LR","A2C","PPO","RF","DT", "SVM"]:
    performance_score["date"] += [date_]
    performance_score["algo"] += [algo]
    if algo == "LR":
      score = lr_score 
    elif algo == "RF":
      score = rf_score
    elif algo == "DT":
      score = dt_score
    elif algo == "A2C":
      score = saliency_score_a2c
    elif algo == "SVM":
      score = svm_score
    else:
      score = saliency_score_ppo
    performance_score["score"] += [score]

performance_score = pd.DataFrame(performance_score)